In [29]:
#!pip3 install dwave-system

In [47]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.composites import EmbeddingComposite
from dwave.system import DWaveSampler, ReverseBatchStatesComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit

In [31]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']),                     
        np.dot(df['Y'], df['LI']),      
        np.dot(df['Y'], df['SEX']),      
        np.dot(df['Y'], df['AOP']),      
    ])     

In [32]:
def make_Hamiltonian(df, t1):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    LI = df['LI'].iloc
    lin_LI = [(LI[i] - 2 * t1) * LI[i] for (i, _) in dup_list]
    quad_LI = [2*LI[i] * LI[j] for (i, j) in comb_list]
    num_LI = t1**2
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[2]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[2]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[3]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[3]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_LI, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_LI, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    
    #print('lin:', lin)
    #print('quad:', quad)
    #print('num:', num)
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [33]:
def find_valid_y(df, num_reads):                                                        
    DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
    
    # dimod sampler
    dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = DWAVE_TOKEN
    )
    
    # インスタンス作成
    #qa_sampler = AutoEmbeddingComposite(dw_sampler)
    qa_sampler = EmbeddingComposite(dw_sampler)
    
    t_list = calc_marginals(df)
    valid_y_list = {}                                                                   
    valid_y_num = {}
    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        #bqm
        bqm = make_Hamiltonian(df, t1)
        
        #method
        #res = qa_sampler.sample(bqm, return_embedding=True, num_reads=num_reads)
        #res = qa_sampler.sample(bqm, chain_strength=3, chain_break_fraction=True, return_embedding=True, num_reads=num_reads)
        res = qa_sampler.sample(bqm, chain_break_fraction=False, num_reads=num_reads)
        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                             
        for y_info in list(res.record):
            #print(y_info)
            if y_info[1] == 0.:
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(list(y_info[0]))                    
                #print('energy0')
                                                      
    return valid_y_list, valid_y_num       

In [41]:
def QA_ReverseAdvanceComposite(df):
    valid_y_list = {}                                                                   
    valid_y_num = {}
    for t1 in range(0, sum(df['LI'])):
        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        
        bqm = make_Hamiltonian(df, t1)

        dw_sampler = DWaveSampler(
            endpoint="https://cloud.dwavesys.com/sapi",
            solver = 'DW_2000Q_6',
            token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
        )

        qa_sampler = EmbeddingComposite(dw_sampler)
        sampler_reverse = ReverseAdvanceComposite(qa_sampler)
        init_samples = {n: y for n, y in enumerate(df['Y'].values.tolist())}

        res = sampler_reverse.sample(bqm,
                                       anneal_schedules=[[[0.0, 1.0], [t, 0.5], [20, 1.0]] for t in (5, 10, 6)],
                                       initial_state=init_samples,
                                       num_reads=10,
                                       reinitialize_state=True)

        for canditate_y_info in list(res.record):
            if canditate_y_info[1]==0.:
                if all(list(canditate_y_info[0]) != p for p in valid_y_list[t1]): 
                    valid_y_list[t1].append(list(canditate_y_info[0]))
                    valid_y_num[t1] += 1
                
    return valid_y_list, valid_y_num

In [54]:
def QA_ReverseBatchStatesComposite(df):
    valid_y_list = {}                                                                   
    valid_y_num = {}
    for t1 in range(0, sum(df['LI'])):
        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        
        bqm = make_Hamiltonian(df, t1)

        dw_sampler = DWaveSampler(
            endpoint="https://cloud.dwavesys.com/sapi",
            solver = 'DW_2000Q_6',
            token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
        )

        qa_sampler = EmbeddingComposite(dw_sampler)
        sampler_reverse = ReverseBatchStatesComposite(qa_sampler)
        init_samples = {n: y for n, y in enumerate(df['Y'].values.tolist())}

        res = sampler_reverse.sample(bqm,
                                     initial_state=init_samples,
                                     anneal_schedule=[[0.0, 1.0], [10.0, 0.5], [20, 1.0]],
                                     num_reads=10,
                                     reinitialize_state=True)

        for canditate_y_info in list(res.record):
            if canditate_y_info[1]==0.:
                if all(list(canditate_y_info[0]) != p for p in valid_y_list[t1]): 
                    valid_y_list[t1].append(list(canditate_y_info[0]))
                    valid_y_num[t1] += 1
                
    return valid_y_list, valid_y_num

In [141]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([0, 1, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]])

[2 2 0 1]
[2 2 1 2]


AssertionError: 

In [72]:
def time_measurement(df):
    sum_time = 0
    annealing_time = 20
    for t1 in range(0, sum(df['LI'])+1):
        timeit_repeat = timeit.repeat("make_Hamiltonian(df, t1)", number=1, repeat=1, globals={"make_Hamiltonian": make_Hamiltonian, "df": df, "t1": t1})
        sum_time += timeit_repeat[0] + annealing_time
    return sum_time

In [73]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads=20)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0

{0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0]], 3: [], 4: [], 5: [], 6: []} {0: 0, 1: 0, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0}


In [64]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]])

[2 2 0 1]
[2 2 0 1]
